In [ ]:
!pip install pulp

!sudo apt-get install coinor-cbc glpk-utils coinor-clp


In [2]:
### kod sprawdza czy instalacja się udała
import pulp

print(pulp.__version__)

2.3.1


In [ ]:
pulp.pulpTestAll()

# Zadania:

1. przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. w blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

Takie zdanie można skleić z części generowanych automatycznie:



In [ ]:
sklejone_produkt_cena = [":".join((produkt, str(cena))) for produkt,cena in zip(["jabłko","banan"],[5,10])]
print(sklejone_produkt_cena)
sklejona_calosc = ", ".join(sklejone_produkt_cena)

print(sklejona_calosc)

['jabłko:5', 'banan:10']
jabłko:5, banan:10


In [3]:
from pulp import *

Definiowanie problemu

In [ ]:
prob = LpProblem("The_Whiskas_Problem",LpMinimize)

x1=LpVariable("ChickenPercent",0)
x2=LpVariable("BeefPercent",0)
x3=LpVariable("Mutton",0)

prob += 0.013*x1 + 0.008*x2 + 0.015*x3, "Total Cost of Ingredients per can"

prob += x1 + x2 + x3 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 + 0.150*x3 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 + 0.110*x3 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 + 0.003*x3 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 + 0.007*x3 <= 0.4, "SaltRequirement"
prob += 0.003*x1 + 0.026*x2 + 0.026*x3 >= 0.8, "VitBRequirement" 

# Smak baraniny
prob += x1 >= 0, "MinChicken"
prob += x2 >= 0, "MinBeef"
prob += x3 >= 20, "MinMutton"

# Rozwiązywanie problemu
prob.writeLP("WhiskasModel.lp")
prob.solve()

print("Status: ", LpStatus[prob.status])

print("Koncowy produkt bedzie kosztowal", value(prob.objective), "I bedzie zawieral: ")
for v in prob.variables():
    print(v.name, "=", v.varValue)

Status:  Optimal
Koncowy produkt bedzie kosztowal 1.173333335 I bedzie zawieral: 
BeefPercent = 33.333333
ChickenPercent = 46.666667
Mutton = 20.0


W powyższym problemie wszytkie składniki odżywcze zdefiniowane były wewnątrz problemu a ich modyfikacja była skomplikowana. 
Poniżej wszystkie parametry poszczególnych skadników zostały wcześniej zgrupowane w poszczególnych słownikach.

W naszym przypadku wydaje się to mało efektywną techniką jednak w momencie kiedy problem staje się bardziej rozbudowany, to takie rozwiązanie może pomóc w szybszej implementacji problemu. 

In [ ]:
from pulp import *
# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL', 'SARDINES']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001,
         'SARDINES': 0.015}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000,
                  'SARDINES': 0.150}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'SARDINES': 0.008}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'SARDINES': 0.000}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'SARDINES': 0.001}

vitBPercent = {'CHICKEN': 0.003, 
               'BEEF': 0.026, 
               'MUTTON': 0.026, 
               'RICE': 0.00, 
               'WHEAT': 0.00, 
               'GEL': 0.00,
               'SARDINES': 0.89}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("The_Whiskas_Problem", LpMinimize)
# A dictionary called 'ingredient_vars' is created to contain the referenced Variables
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0)
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"

# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([vitBPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 0.8, "VitBRequirement"

#Wersja szalone sardynki
prob += ingredient_vars['BEEF'] <= 5, "LessBEEF"
prob += ingredient_vars['SARDINES'] >=20, "MinSardines"

prob.solve()

print("Status: ", LpStatus[prob.status])

print("Koncowy produkt bedzie kosztowal", value(prob.objective), "I bedzie zawieral: ")
for v in prob.variables():
    print(v.name, "=", v.varValue)

Status:  Optimal
Koncowy produkt bedzie kosztowal 0.8519090949999999 I bedzie zawieral: 
Ingr_BEEF = 5.0
Ingr_CHICKEN = 0.0
Ingr_GEL = 26.454545
Ingr_MUTTON = 48.545455
Ingr_RICE = 0.0
Ingr_SARDINES = 20.0
Ingr_WHEAT = 0.0
